In [1]:
import re
import pandas as pd
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split

In [2]:
yahoo = pd.concat([pd.read_csv(f,index_col=False) for f in glob('../data/yahoo*')])

In [3]:
yahoo = yahoo.reset_index(drop=True)
yahoo.tail(10)

,label,Title,Title_URL,Image,Comment,newsfeed_item_media
5508,0,令和初の新王者・中谷潤人が出身地・三重県の高校生ボクサーらと交流「帰省できてよかった」,https://headlines.yahoo.co.jp/hl?a=20201224-00...,https://amd-pctr.c.yimg.jp/r/iwiz-amd/20201224...,NaN,中日スポーツ
5509,0,日テレ、浦和が決勝進出 皇后杯サッカー,https://headlines.yahoo.co.jp/hl?a=20201224-00...,https://amd-pctr.c.yimg.jp/r/iwiz-amd/20201224...,NaN,時事通信
5510,0,川崎・三笘ブレークの陰で…ふがいない自分が「悔しい」相馬勇紀、代表生き残りへ見えた“道筋”【...,https://headlines.yahoo.co.jp/hl?a=20201224-00...,https://amd-pctr.c.yimg.jp/r/iwiz-amd/20201224...,NaN,中日スポーツ
5511,0,【RISE】若手女子4名によるトーナメント開催、優勝者は王者・紅絹の挑戦権決定戦へ進出,https://headlines.yahoo.co.jp/hl?a=20201224-00...,https://amd-pctr.c.yimg.jp/r/iwiz-amd/20201224...,NaN,イーファイト
5512,0,ロッテ獲得発表 アデイニー・エチェバリアという男 内野の超人的な守備にイチロー氏も感嘆,https://headlines.yahoo.co.jp/hl?a=20201224-00...,https://amd-pctr.c.yimg.jp/r/iwiz-amd/20201224...,NaN,スポニチアネックス
5513,0,内藤哲也と飯伏幸太の熱愛にジェイ・ホワイトが歯噛み！「1・4でしようぜ。最高の俺らのプロレス！」,https://headlines.yahoo.co.jp/hl?a=20201224-00...,https://amd-pctr.c.yimg.jp/r/iwiz-amd/20201224...,NaN,バトル・ニュース
5514,0,讃岐が大卒ルーキー7名含む、9選手との契約更新を発表,https://headlines.yahoo.co.jp/hl?a=20201224-00...,https://amd-pctr.c.yimg.jp/r/iwiz-amd/20201224...,NaN,超WORLDサッカー！
5515,0,“ネットから情報”取捨選択の難しさ…動作解析の第一人者が力説「指導者も学ぶ姿勢を」,https://headlines.yahoo.co.jp/hl?a=20201224-01...,https://amd-pctr.c.yimg.jp/r/iwiz-amd/20201224...,NaN,Full-Count
5516,0,【名珍場面2020】渋野日向子が南国ビーチで…ゴルフ仲間らと“ワンピース風”絆写真に反響6万超,https://headlines.yahoo.co.jp/article?a=202012...,https://amd-pctr.c.yimg.jp/r/iwiz-amd/20201224...,NaN,THE ANSWER
5517,0,元ヤクルト五十嵐氏インスタ開設 穏やかな笑顔写真,https://headlines.yahoo.co.jp/hl?a=20201224-22...,https://amd-pctr.c.yimg.jp/r/iwiz-amd/20201224...,NaN,日刊スポーツ


In [4]:
# Remove new line character
to_hankaku = {chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}

def parse(x):
    x = re.sub('[『』〝〟【】「」“”．《》＜＞\[\]〔〕＝]','',x) #『』を除去
    x = re.sub('[&＆・×]','と', x)
    x = re.sub('〜','から',x)
    x = re.sub('　','',x) # 無駄な空白を除去
    x = x.translate(str.maketrans(to_hankaku)) # 全角→半角変換
    x = x.lower() # 大文字→小文字
    
    x = re.sub('[0-9]+','0',x) # 番号を全て0にする
    x = re.sub('[!?]','。',x)
    x = re.sub('…','。',x)
    x = re.sub('。+','。',x)
    
    return x

In [5]:
yahoo['Title'] = yahoo['Title'].map(lambda x: parse(x)).values

In [6]:
yahoo.head(50)

,label,Title,Title_URL,Image,Comment,newsfeed_item_media
0,0,唐田えりか、事務所仕事のかたわら目指す声優デビューへの道筋,https://headlines.yahoo.co.jp/article?a=202012...,https://news-pctr.c.yimg.jp/q3eRX5yhdvIzm3wm4v...,NaN,週刊女性PRIME
1,0,松坂桃李と戸田恵梨香の結婚をノーマークにさせた綾瀬はるかのマンション出入り,https://headlines.yahoo.co.jp/article?a=202012...,https://news-pctr.c.yimg.jp/q3eRX5yhdvIzm3wm4v...,NaN,週刊女性PRIME
2,1,宮崎大輔、離婚成立を認めた。 暴行逮捕騒動の0歳下恋人と今も一緒に住んでます,https://headlines.yahoo.co.jp/article?a=202012...,https://news-pctr.c.yimg.jp/q3eRX5yhdvIzm3wm4v...,NaN,SmartFLASH
3,0,辻希美、長女と長男と次男の晴れ舞台に感動涙がでました皆頑張ったね,https://headlines.yahoo.co.jp/hl?a=20201221-00...,https://news-pctr.c.yimg.jp/q3eRX5yhdvIzm3wm4v...,NaN,E-TALENTBANK
4,0,木村祐一の妻と西方凌、0歳長女と生後0か月長男を公開パパ似ですね可愛い姉弟,https://headlines.yahoo.co.jp/hl?a=20201221-12...,https://news-pctr.c.yimg.jp/q3eRX5yhdvIzm3wm4v...,NaN,スポーツ報知
5,1,上方落語協会会長と笑福亭仁智こん平さんの死を悼む林家をまとめ。本当に大変な功績,https://headlines.yahoo.co.jp/hl?a=20201221-02...,https://news-pctr.c.yimg.jp/q3eRX5yhdvIzm3wm4v...,NaN,東スポWeb
6,0,記者もお手上げのインドアライフ芸能マスコミを驚かせた電撃婚の戸田恵梨香、松坂桃李,https://headlines.yahoo.co.jp/hl?a=20201221-00...,https://news-pctr.c.yimg.jp/q3eRX5yhdvIzm3wm4v...,NaN,夕刊フジ
7,0,加護亜依 家族をテレビ初公開、夫が涙した理由とは。長女は辻ちゃん派,https://headlines.yahoo.co.jp/hl?a=20201221-00...,https://news-pctr.c.yimg.jp/q3eRX5yhdvIzm3wm4v...,NaN,スポニチアネックス
8,0,使い勝手がよすぎるから教えたい。ダイソーサッと取り出せる手さげ袋が凄い。,https://headlines.yahoo.co.jp/article?a=202012...,https://news-pctr.c.yimg.jp/q3eRX5yhdvIzm3wm4v...,NaN,レタスクラブニュース
9,0,アレク赤ちゃんのテヘペロ写真撮った。フォロワー絶賛可愛すぎる,https://headlines.yahoo.co.jp/hl?a=20201221-00...,https://news-pctr.c.yimg.jp/q3eRX5yhdvIzm3wm4v...,NaN,デイリースポーツ


In [7]:
yahoo = yahoo.loc[:,['label','Title']]
yahoo.drop_duplicates()

,label,Title
0,0,唐田えりか、事務所仕事のかたわら目指す声優デビューへの道筋
1,0,松坂桃李と戸田恵梨香の結婚をノーマークにさせた綾瀬はるかのマンション出入り
2,1,宮崎大輔、離婚成立を認めた。 暴行逮捕騒動の0歳下恋人と今も一緒に住んでます
3,0,辻希美、長女と長男と次男の晴れ舞台に感動涙がでました皆頑張ったね
4,0,木村祐一の妻と西方凌、0歳長女と生後0か月長男を公開パパ似ですね可愛い姉弟
...,...,...
5512,0,ロッテ獲得発表アデイニーとエチェバリアという男内野の超人的な守備にイチロー氏も感嘆
5513,0,内藤哲也と飯伏幸太の熱愛にジェイとホワイトが歯噛み。0と0でしようぜ。最高の俺らのプロレス。
5514,0,讃岐が大卒ルーキー0名含む、0選手との契約更新を発表
5515,0,ネットから情報取捨選択の難しさ。動作解析の第一人者が力説指導者も学ぶ姿勢を


In [8]:
y, X = yahoo['label'], yahoo['Title']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [9]:
train = pd.DataFrame({'label': y_train, 'text': X_train})
test = pd.DataFrame({'label': y_test, 'text': X_test})

In [10]:
len(train)

4138

In [11]:
len(test)

1380

In [12]:
test.to_csv('../data/test.txt', sep='\t', index=False, header=False)
train.to_csv('../data/train.txt', sep='\t', index=False, header=False)

In [13]:
test.label.mean()

0.2326086956521739

In [14]:
train.label.mean()

0.2247462542290962